In [6]:
import numpy as np
import pandas as pd
import sys
import os
# Define the base path depending on the environment
nameCompetition = "playground-series-s4e12"
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = "/content/drive/MyDrive/data/kaggle/" + nameCompetition
    print("Google Colab detected. Base path set to:", base_path)

elif "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
    base_path = "/kaggle/input/" + nameCompetition
    print("Kaggle environment detected. Base path set to:", base_path)
else:
    base_path = "/mnt/g/Mon Drive/data/kaggle/" + nameCompetition
    print("Local environment detected. Base path set to:", base_path)
# List all files in the base path directory
if os.path.exists(base_path):
    print(f"Contents of the directory '{base_path}':")
    for dirname, _, filenames in os.walk(base_path):
        for filename in filenames:
            print(os.path.join(dirname, filename))
else:
    print(f"The directory '{base_path}' does not exist.")


Local environment detected. Base path set to: /mnt/g/Mon Drive/data/kaggle/playground-series-s4e12
Contents of the directory '/mnt/g/Mon Drive/data/kaggle/playground-series-s4e12':
/mnt/g/Mon Drive/data/kaggle/playground-series-s4e12/test.csv
/mnt/g/Mon Drive/data/kaggle/playground-series-s4e12/train.csv
/mnt/g/Mon Drive/data/kaggle/playground-series-s4e12/sample_submission.csv


In [ ]:
df_train = pd.read_csv(base_path + "/train.csv")
df_test = pd.read_csv(base_path + "/test.csv")

# Context

# EDA

# preprocessing

# Model